In [11]:
import pandas as pd
import numpy as np
from scipy.spatial.transform import Rotation as R
import open3d as o3d
import time
import os

In [12]:
def load_csv(file_path):
    try:
        df = pd.read_csv(file_path, sep=";")
        print(df.shape)
        if df.empty:
            print("The CSV file is empty.")
            return None
        return df
    except pd.errors.EmptyDataError:
        print("No data in the CSV file.")
        return None
    except pd.errors.ParserError:
        print("Error parsing the CSV file.")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None


In [13]:
def get_camera_matrices(camera_path):
    camdf = pd.read_csv(camera_path)
    camdf = camdf.set_index(camdf['Property'])
    camdf = camdf.drop(['Property'], axis = 1)

    fov = round(float(camdf.loc['fov']['Value']), 2)
    location_x = round(float(camdf.loc['location_x']['Value']), 2)
    location_y = round(float(camdf.loc['location_y']['Value']), 2)
    location_z = round(float(camdf.loc['location_z']['Value']), 2)
    rotation_euler_x = round(float(camdf.loc['rotation_euler_x']['Value']), 2)
    rotation_euler_y = round(float(camdf.loc['rotation_euler_y']['Value']), 2)
    rotation_euler_z = round(float(camdf.loc['rotation_euler_z']['Value']), 2)
    resolution_x = round(float(camdf.loc['resolution_x']['Value']), 2)
    resolution_y = round(float(camdf.loc['resolution_y']['Value']), 2)

    cx = resolution_x / 2
    cy = resolution_y / 2
    fx = cx / np.tan(np.radians(fov / 2))
    fy = fx

    K = np.array([
        [fx, 0, cx],
        [0, fy, cy],
        [0, 0, 1]
    ])

    rotation_euler = np.radians([rotation_euler_x, rotation_euler_y, rotation_euler_z])
    R_matrix = R.from_euler('xyz', rotation_euler).as_matrix()

    t = np.array([location_x, location_y, location_z]).reshape((3, 1))

    extrinsic = np.hstack((R_matrix, t))
    extrinsic = np.vstack((extrinsic, np.array([0, 0, 0, 1])))

    return int(resolution_x), int(resolution_y), K, extrinsic

In [14]:
def get_point_cloud(file_path, noise = False):
    df = load_csv(file_path)
    if noise: coord = ['X', 'Y', 'Z']
    else: coord = ['X_noise', 'Y_noise', 'Z_noise']

    points = df[coord].values
    colors = df[['red', 'green', 'blue']].values

    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(points)
    point_cloud.colors = o3d.utility.Vector3dVector(colors)

    return point_cloud

In [15]:
def visualize_point_cloud(file_path, camera_path, noise = False):

    pcd = get_point_cloud(file_path, noise)

    resx, resy, icm, ecm = get_camera_matrices(camera_path)
    
    vis = o3d.visualization.Visualizer()
    vis.create_window(window_name='Virtual Information Space', width=950, height=1080, left=0, top=35)
    vis.add_geometry(pcd)

    cameraLines = o3d.geometry.LineSet.create_camera_visualization(view_width_px=resx, view_height_px=resy, intrinsic = icm, extrinsic = ecm)
    vis.add_geometry(cameraLines)

    last_modified_time = os.path.getmtime(file_path)

    print(time.ctime(last_modified_time))

    try:
        while True:
            current_modified_time = os.path.getmtime(file_path)
            
            if current_modified_time != last_modified_time:
                temp = get_point_cloud(file_path, noise)
                if temp is not None: 
                    pcd = temp
                    print(pcd.points)
                    vis.clear_geometries()
                    vis.add_geometry(pcd)
                last_modified_time = current_modified_time
            
            vis.update_geometry(pcd)
            vis.poll_events()
            vis.update_renderer()

    except KeyboardInterrupt:
        print("Stopped by user")
        
    vis.destroy_window()

file_path = 'C:\\Programming\\Fourth Semester\\PointVIS\\PointVIS\\assets\\output\\toilet4_frames_17_to_17.csv'
camera_path = 'C:\\Programming\\Fourth Semester\\PointVIS\\PointVIS\\assets\\output\\camera4.csv'
visualize_point_cloud(file_path = file_path, noise = False, camera_path = camera_path)
print("Hello")

(45061, 14)
Thu Jun 13 01:47:23 2024
Stopped by user
Hello


In [1]:
import open3d 

WIDTH = 1280
HEIGHT = 720

vizualizer = open3d.visualization.Visualizer()
vizualizer.create_window()
vizualizer.create_window(window_name='Virtual Information Space', width=950, height=1080, left=0, top=35)

standardCameraParametersObj = vizualizer.get_view_control().convert_to_pinhole_camera_parameters()
# cameraLines = open3d.geometry.LineSet.create_camera_visualization(intrinsic=standardCameraParametersObj.intrinsic, extrinsic=standardCameraParametersObj.extrinsic)
cameraLines = open3d.geometry.LineSet.create_camera_visualization(view_width_px=WIDTH, view_height_px=HEIGHT, intrinsic=standardCameraParametersObj.intrinsic.intrinsic_matrix, extrinsic=standardCameraParametersObj.extrinsic)
vizualizer.add_geometry(cameraLines)

# vizualizer.run()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


True

# Trail and Error

In [3]:
standardCameraParametersObj.intrinsic.intrinsic_matrix

array([[913.65680099,   0.        , 474.5       ],
       [  0.        , 913.65680099, 527.        ],
       [  0.        ,   0.        ,   1.        ]])

In [4]:
import numpy as np
from scipy.spatial.transform import Rotation as R

# Camera parameters
fov = 90.00000250447816
location_x = 6.241810321807861
location_y = -37.952274322509766
location_z = 0.8069829940795898
rotation_euler_x = 78.08459406341835
rotation_euler_y = -0.0018460653965008975
rotation_euler_z = 454.89505200618805
resolution_x = 640
resolution_y = 480

# Calculate the intrinsic matrix K
cx = resolution_x / 2
cy = resolution_y / 2
fx = cx / np.tan(np.radians(fov / 2))
fy = fx

K = np.array([
    [fx, 0, cx],
    [0, fy, cy],
    [0, 0, 1]
])

# Convert Euler angles to rotation matrix
rotation_euler = np.radians([rotation_euler_x, rotation_euler_y, rotation_euler_z])
R_matrix = R.from_euler('xyz', rotation_euler).as_matrix()

# Translation vector
t = np.array([location_x, location_y, location_z]).reshape((3, 1))

# Form the extrinsic matrix
extrinsic = np.hstack((R_matrix, t))
extrinsic = np.vstack((extrinsic, np.array([0, 0, 0, 1])))

# Print results
print("Intrinsic matrix K:\n", K)
print("Extrinsic matrix E:\n", extrinsic)


Intrinsic matrix K:
 [[319.99998601   0.         320.        ]
 [  0.         319.99998601 240.        ]
 [  0.           0.           1.        ]]
Extrinsic matrix E:
 [[-8.53308796e-02 -2.05711539e-01  9.74885328e-01  6.24181032e+00]
 [ 9.96352668e-01 -1.76494456e-02  8.34856702e-02 -3.79522743e+01]
 [ 3.22199194e-05  9.78453504e-01  2.06467284e-01  8.06982994e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [11]:
K

array([[319.99998601,   0.        , 320.        ],
       [  0.        , 319.99998601, 240.        ],
       [  0.        ,   0.        ,   1.        ]])

: 